### Operating principles
* read in dump.json as mazvy gives it. This will allow him to take responsiblity for updating or appending to dump.json.
* Create new columns in dedicated cells; 
* do not rewrite or drop original columns (to facilitate bug testing; unless memory becomes a problem)


In [5]:
noFigs=True

In [6]:
import plotly.express as px
import pandas as pd

In [7]:
df = pd.read_json('dump.json')
df.columns

Index(['id', 'user', 'body', 'topics', 'datetime', 'visibility',
       'profilePicture', 'permalink'],
      dtype='object')

In [8]:
df['threadY'] = (df.id - round(df.id) * 100000)
df['name'] = [user.replace('(Legacy) ', '') for user in df['user']] 
df['text'] = [tup.name + ' ' + str(tup.datetime)[:19] for tup in df.itertuples()] #Jon Schull 2013-07-10 20:01:06


In [9]:
from IPython.display import Markdown
from markdownOfThread import markdownOfThread
#Markdown(markdownOfThread(df,9))

In [10]:
df['threadY'] = (df.id - round(df.id) * 100000)
df['name'] = [user.replace('(Legacy) ', '') for user in df['user']] 
df['text'] = [tup.name + ' ' + str(tup.datetime)[:19] for tup in df.itertuples()] # Jon Schull 2013-07-10 20:01:06


In [11]:
df['IDcolor']=round(df['id']%10) #ten discrete colors.  
df['IDcolor'] = df['IDcolor'].astype(str) #needed for discrete colorscale?
df['threadPos'] = 0.01 + 100*(df.id - round(df.id))  #for sizing.  Grows with the post's position in the thread with a minimum size of 0.01

# ThreadsFig, colored by threadID

In [12]:
fig = px.scatter(df, x='datetime', y='id', color='IDcolor', size='threadPos', height=800, hover_name='name', opacity=0.4, template='plotly_dark')
fig.update_layout(showlegend=False, 
                  title_text='Threads: Posts over Time, Colored by Thread, Sized by Position in Thread')
fig.update_yaxes(title_text='Successive Threads')
threadsFig = fig
if noFigs: threadsFig

# PeopleFig

In [13]:
#make UniqueNameNumbers
#names = df['user']
uniqueNames = []
for name in df.name:
    name = name.replace('(Legacy) ', '')
    if name not in uniqueNames:
        uniqueNames.append(name)
        
from collections import OrderedDict
uniqueNameNumbers = UNN = OrderedDict()
for i,name in enumerate( uniqueNames ):
    UNN[name] = i

UNNs = []
for name in df.name:
        UNNs.append(UNN[ name.replace('(Legacy) ','')])
#UNNs

In [14]:
df['UNN'] = UNNs
df['UNNcolor'] = df['UNN'] % 10
df['UNNcolor'] = df['UNNcolor'].astype(str)
df['smThreadPos'] = df.threadPos / 1000

In [15]:
fig = px.scatter(df,x='datetime', y='UNN', color='IDcolor',size='smThreadPos', height=800, hover_name='text', opacity=0.3, template='plotly_dark')
fig.update_layout(showlegend=False, 
                  title_text='People: Posts over Time, Colored by Thread')
fig.update_yaxes(title_text='People ordered by first post')

peopleFig = fig
if not noFigs: peopleFig

In [16]:
fig = px.scatter(df, y='UNN', x='id',color='IDcolor', height=1200, size='smThreadPos', hover_name='text', opacity=0.3, template='plotly_dark')
fig.update_layout(showlegend=False, 
                  title_text="""Horizontal bands are People. Vertical columns of the same color are threads.""")
fig.update_yaxes(title_text='People ordered by first post')
fig.update_xaxes(title_text='Successive Threads over time.')

bothFig = fig
if not noFigs: bothFig